
# Genie API 基本編（1問だけ投げてみる）

このノートブックでは、次の流れを確認します。

1. Genie に自然言語の質問を送る（会話開始）
2. 処理が完了するまで待つ
3. 生成された回答テキストと SQL を確認する

In [0]:
%run ./_init_genie_api

### 1. 生の final_message を見てみよう！

In [0]:
# 質問をセット
question = "直近30日間の売上トップ10の商品を教えて"
print("=== 質問 ===")
print(question)

In [0]:
import json

# 会話開始
start_resp = start_conversation(question)
conversation_id = start_resp.get("conversation", {}).get("id")
message_id = start_resp.get("message", {}).get("id")

print("\n\n=== 会話情報 ===")
print(f"conversation_id: {conversation_id}")
print(f"message_id     : {message_id}")

# 完了まで待つ
final_message = wait_until_completed(
    conversation_id,
    message_id,
    timeout_sec=120,    # 2分でタイムアウト
    interval_sec=5,
)

print("\n\n=== Genie からの最終メッセージ(JSON 全体) ===")
print(json.dumps(final_message, ensure_ascii=False, indent=2))

### 2. final_message から必要な要素だけ取り出そう！

In [0]:
status = final_message.get("status")
content = final_message.get("content")  # ユーザーの質問
attachments = final_message.get("attachments", [])

answer_text = None              # 説明テキスト（description）
generated_sql = None            # 生成されたSQL
attachment_id_for_query = None  # 結果取得用 attachment_id
suggested_questions = []        # 提案質問リスト

for i, att in enumerate(attachments):
    print(f"--- attachment #{i} --- keys: {list(att.keys())}")

    query_part = att.get("query")
    suggested_part = att.get("suggested_questions")

    # 生成SQL＋説明テキスト＋結果用 attachment_id
    if query_part and generated_sql is None:
        generated_sql = query_part.get("statement") or query_part.get("query")
        answer_text = query_part.get("description")   # 回答テキスト
        attachment_id_for_query = att.get("attachment_id") or query_part.get("attachment_id")

    # 提案質問（あれば配列として回収）
    if suggested_part:
        qs = suggested_part.get("questions") or []
        suggested_questions.extend(qs)

print("=== 1) メッセージ全体のステータス ===")
print(status)

print("\n=== 2) ユーザーの質問 (content) ===")
print(content)

print("\n=== 3) Genie の回答（説明テキスト / description） ===")
print(answer_text)

print("\n=== 4) Genie が生成した SQL ===")
print(generated_sql)

print("\n=== 5) クエリ結果用 attachment_id ===")
print(attachment_id_for_query)

print("\n=== 6) 提案質問 (suggested_questions) ===")
for q in suggested_questions:
    print(f"  - {q}")

### 3. SQL実行結果データセットを取り出してみよう！

In [0]:
if attachment_id_for_query:
    result_json = get_query_result(
        conversation_id=conversation_id,
        message_id=message_id,
        attachment_id=attachment_id_for_query,
    )

    # JSON取得 → DataFrame変換
    df_result = genie_query_result_to_df(result_json)

    print("=== クエリ結果（先頭20行） ===")
    display(df_result.limit(20))
else:
    print("attachment_id_for_query がありませんでした。")